## Load required libraries


In [2]:
!pip install rasterio requests numpy matplotlib

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------- -------------------------- 20.5/61.0 kB ? eta -:--:--
     ------------------------------- ------ 51.2/61.0 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 546.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/165.5 kB ? eta -:--:--
     -------------------------------------  163.8/165.5 kB 5.0 MB/s eta 0:00:01
     -------------------------------------- 165.5/165.5 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.3 MB ? eta -:--:--
    --------------------------------------- 0.4/24.3 MB 26.9 MB/s eta 0:00:01
   - -------------------------------------- 1.0/24.3 MB 13.3 MB/s eta 0:00:02
   - -------------------------------------- 1.0/24.3 MB 13.3 MB/s eta 0:00:02
   - -------------------------------------- 1.0/24.3 MB 13.3 MB/s eta 0:00:02
   - ---

In [3]:
import rasterio
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.enums import Resampling
import numpy as np

## get the URL for the DEM 

This dem is stored on amazon web services and is accessible for download as a url, if you put the link below in your browser it will download the data, if you prefer to have a local copy, that will save this next step from taking several minutes. then just replace the url with the path on your computer.


In [ ]:
# URL of your DEM.tif file in the S3 bucket
url = 'https://atur-karst.s3.us-east-2.amazonaws.com/MMRK1m.tif'

# Download the file
response = requests.get(url)
response.raise_for_status()

# Open the file with rasterio
with rasterio.open(BytesIO(response.content)) as dataset:
    dem_data = dataset.read(1, resampling=Resampling.bilinear)
    profile = dataset.profile
    print(profile)

# plot the DEM

next we will plot the loaded DEM as a hillshade using rasterio and matplot lib

In [ ]:
# Calculate hillshade
    azimuth = 315
    angle_altitude = 45
    
    x, y = np.gradient(dem_data)
    slope = np.pi/2. - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    azimuth_rad = azimuth * np.pi / 180.
    altitude_rad = angle_altitude * np.pi / 180.
    
    hillshade = np.sin(altitude_rad) * np.sin(slope) + \
                np.cos(altitude_rad) * np.cos(slope) * \
                np.cos((azimuth_rad - np.pi/2.) - aspect)
    hillshade = hillshade * 255
    hillshade = hillshade.astype(np.uint8)

# Plot the hillshade
plt.figure(figsize=(10, 10))
plt.imshow(hillshade, cmap='gray')
plt.title('Hillshade')
plt.axis('off')
plt.show()